In [3]:
# Imports dans un autre fichier
from imports import *

In [13]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL = 'SELECT "averageRating", "startYear", "runtimeMinutes" from principal.title_basics join principal.title_ratings on principal.title_basics."tconst" = principal.title_ratings."tconst"  limit 50000;'
df = pd.read_sql(SQL, engine)

df

,averageRating,startYear,runtimeMinutes
0,9.3,2004.0,45.0
1,6.3,2019.0,41.0
2,6.5,2019.0,29.0
3,5.9,2019.0,37.0
4,8.1,2004.0,43.0
...,...,...,...
49995,9.0,2008.0,42.0
49996,7.3,2008.0,40.0
49997,8.4,2019.0,112.0
49998,7.3,2008.0,49.0


In [14]:
engine.dispose()

In [18]:
for column in ["averageRating", "startYear", "runtimeMinutes"]:
    df[column].fillna(df[column].mean(), inplace=True)

# Target
y = df['averageRating']
# Features
X = df[['startYear','runtimeMinutes']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Séparation de type des données
preparation = ColumnTransformer(
    transformers=[
        ('data_num', RobustScaler(), ["startYear", "runtimeMinutes"])
])

# Pipeline
pipel = Pipeline(steps=[
    ('scaling', preparation),
    ('modelisation', DecisionTreeRegressor())
])

params = {
    'modelisation': [DecisionTreeRegressor()]
}
scorer = {'mean_squared_error': mean_squared_error}

searchCV = GridSearchCV(pipel, params, scoring='neg_mean_squared_error', refit=True)

searchCV.fit(X_train, y_train)
y_pred = searchCV.predict(X_test)
print("Score training:", -searchCV.best_score_)
print("Score recall test:", recall_score(y_test, y_pred, average=None))
print("Le modèle de ML utilisé c'est le", searchCV.best_params_['modelisation'])

/tmp/ipykernel_13972/1074364059.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mean(), inplace=True)


Score training: 1.7204828560535173


ValueError: continuous is not supported